# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0         faya  18.3851   42.4509     22.06        35          21   
1        1      farsund  58.0948    6.8047     13.30       100           0   
2        2  new norfolk -42.7826  147.0587     11.72        58          12   
3        3    jamestown  42.0970  -79.2353      5.77        77         100   
4        4      lanzhou  36.0564  103.7922     14.53        48          59   

   Wind Speed Country        Date  
0        2.60      SA  1666108228  
1        7.65      NO  1666108228  
2        1.34      AU  1666108230  
3        9.77      US  1666107934  
4        1.20      CN  1666108230

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot1 = city_data_df.hvplot.points('Lng','Lat',
                                        geo =True, tiles='OSM',
                                        size = 'Humidity',
                                        color = 'City')
# Display the map
# YOUR CODE HERE
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_df = city_data_df[(city_data_df['Max Temp']>=20)].dropna()

# Display sample data
# YOUR CODE HERE
city_data_df.head()

City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0         0          faya  18.3851   42.4509     22.06        35          21   
7         7     carnarvon -24.8667  113.6333     20.02        76           0   
9         9  saint-pierre -21.3393   55.4781     22.82        68           0   
10       10      colorado -22.8375  -51.9731     20.36        86          96   
12       12        trairi  -3.2778  -39.2689     31.07        51          34   

    Wind Speed Country        Date  
0         2.60      SA  1666108228  
7         7.46      AU  1666108232  
9         5.14      RE  1666108236  
10        4.69      BR  1666107965  
12        8.09      BR  1666108038

### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame.copy(city_data_df)
hotel_df = hotel_df(hotel_df['City'],['Lat'],['Lng']['Max Temp'],['Humidity'],['Country'])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

# Display sample data


<>:3: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:3: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_2543/955664085.py:3: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  hotel_df = hotel_df(hotel_df['City'],['Lat'],['Lng']['Max Temp'],['Humidity'],['Country'])
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_2543/955664085.py:3: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  hotel_df = hotel_df(hotel_df['City'],['Lat'],['Lng']['Max Temp'],['Humidity'],['Country'])
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_2543/955664085.py:3: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  hotel_df = hotel_df(hotel_df['City'],['Lat'],['Lng']['Max Temp'],['Humidity'],['Country'])
/var

TypeError: list indices must be integers or slices, not str

In [ ]:
data =  {'City': city_data_df['City'],
         'Country': city_data_df['Country'],
         'Lat': city_data_df['Lat'],
         'Lng': city_data_df['Lng'],
         'Humidity': city_data_df['Humidity']}

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "limit":1,
    "apiKey":geoapify_key,
}

 # Set base URL
BASE_URL = "https://api.geoapify.com/v2/places"

In [11]:
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # https://api.geoapify.com/v2/places?categories=public_transport.subway&
    # filter=circle:-0.07071648508463113,51.50848194136378,1000&
    # bias=proximity:-0.07071648508463113,51.50848194136378&
    # limit=20&apiKey=YOUR_API_KEY

    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Make and API request using the params dictionaty
    response = requests.get(BASE_URL, params=params)
    # print(response.status_code, response.url)
    
    # Convert the API response from JSON format to Python
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


faya - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
trairi - nearest hotel: Pousada Villa Aurora
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: Chez Bianca & Benoit
bambous virieux - nearest hotel: Otentic Eco Tent Experience
saint-louis - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
naberera - nearest hotel: No hotel found
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
sao joao da barra - nearest hotel: Pousada Mediterrâneo
kapaa - nearest hotel: Pono Kai Resort
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
hilo - nearest hotel: Dolphin Bay Hotel
businga - nearest hotel: Hôtel Roho
lasa - nearest hotel: Hotel "Drei Kreuz"
vaini - nearest hotel: Keleti Internatio

City Country      Lat       Lng  Humidity Hotel  \
0            faya      SA  18.3851   42.4509        35   NaN   
7       carnarvon      AU -24.8667  113.6333        76   NaN   
9    saint-pierre      RE -21.3393   55.4781        68   NaN   
10       colorado      BR -22.8375  -51.9731        86   NaN   
12         trairi      BR  -3.2778  -39.2689        51   NaN   
..            ...     ...      ...       ...       ...   ...   
567     sangamner      IN  19.5667   74.2167        85   NaN   
568     road town      VG  18.4167  -64.6167        70   NaN   
569        ranong      TH   9.9658   98.6348        93   NaN   
574     canchungo      GW  12.0672  -16.0333        61   NaN   
576          poum      NC -20.2333  164.0167        79   NaN   

               Hotel Name  
0          No hotel found  
7          No hotel found  
9            Tropic Hotel  
10    Servilha Park Hotel  
12   Pousada Villa Aurora  
..                    ...  
567           Joshi Hotel  
568        No hotel found  
569     Rattana Sin Hotel  
574         Hotel Pereira  
576        No hotel found  

[259 rows x 7 columns]

In [15]:
saint_lousi_filter_series = hotel_df['City'] == 'saint-louis'
hotel_df[saint_lousi_filter_series]

City Country      Lat   Lng  Humidity Hotel
29  saint-louis      SN  16.3333 -15.0        24   NaN

In [6]:
# Set parameters to search for a hotel
radius = 10000
latitude = hotel_df['Lat']
longitude = hotel_df['Lng']
bias = f'proximity:'
params = {"categories":'accommodation.hotel',
    "limit":1,
    "bias":f'proximity:{latitude},{longitude},{radius}',
    "apiKey":geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = f'{base_url},{params}'
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


City Country      Lat       Lng  Humidity  \
45            kapaa      US  22.0752 -159.3190        84   
51             hilo      US  19.7297 -155.0900        83   
63            banda      IN  25.4833   80.3333        52   
81    makakilo city      US  21.3469 -158.0858        81   
152         kahului      US  20.8947 -156.4700        60   
197             gat      IL  31.6100   34.7642       100   
211          laguna      US  38.4210 -121.4238        79   
240     tikaitnagar      IN  26.9500   81.5833        59   
265     san quintin      MX  30.4833 -115.9500        74   
340   santa rosalia      MX  27.3167 -112.2833        56   
363          narwar      IN  25.6500   77.9000        55   
375    port hedland      AU -20.3167  118.5667        73   
381       roebourne      AU -20.7833  117.1333        65   
391  saint-francois      FR  46.4154    3.9054        57   
409       capoterra      IT  39.1763    8.9718        71   
421          stolac      BA  43.0844   17.9575        68   
516  guerrero negro      MX  27.9769 -114.0611        68   

                                Hotel Name  
45                         Pono Kai Resort  
51                       Dolphin Bay Hotel  
63                           #acnindiafy21  
81   Embassy Suites by Hilton Oahu Kapolei  
152                     Maui Seaside Hotel  
197                         No hotel found  
211           Holiday Inn Express & Suites  
240                         No hotel found  
265                         Jardines Hotel  
340                         Hotel del Real  
363                         No hotel found  
375                    The Esplanade Hotel  
381                         No hotel found  
391                              Chez Lily  
409                             Rosa Hotel  
421                                Bregava  
516                      Plaza sal paraiso

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)